In [ ]:
from tqdm import tqdm
import numpy as np

In [ ]:
def load_supertags(path):

    file = open(path, 'r')
    word_to_tag = {}

    for line in tqdm(file):

        word, tag = line.split(" ")
        if tag[-1] == "\n": tag = tag[:-1]

        word_to_tag[word.lower()] = tag
    
    file.close()
    return word_to_tag

# Unigrams

## Data Loading

In [ ]:
def load_unigrams(path):

    file = open(path, 'r')
    word_to_occurrence = {}

    for line in tqdm(file):

        words = line.lower().split(" ")
        
        for word in words:

            if word[-1] == "\n": word = word[:-1]

            if word in word_to_occurrence.keys():
                word_to_occurrence[word] += 1
            else:
                word_to_occurrence[word] = 1

    return word_to_occurrence

In [ ]:
def update_tags_and_create_tag_to_wo(word_to_tag, word_to_occurrence):

    tag_to_words_with_occurrence = {}

    for word in tqdm(word_to_occurrence.keys()):

        if word not in word_to_tag.keys():
            word_to_tag[word] = '^' + word

        tag = word_to_tag[word]
        
        if tag in tag_to_words_with_occurrence.keys():
            tag_to_words_with_occurrence[tag].append([word, word_to_occurrence[word]])
        else:
            tag_to_words_with_occurrence[tag] = [[word, word_to_occurrence[word]]]
    
    return tag_to_words_with_occurrence, word_to_tag

In [ ]:
path_supertags = '/content/drive/My Drive/Colab Notebooks/NLP/Dane/Copy of supertags.txt'
path_corpus = '/content/polish_corpora.txt'

word_to_tag = load_supertags(path_supertags)
word_to_occurrence = load_unigrams(path_corpus)
tag_to_words_with_occurrence, word_to_tag = update_tags_and_create_tag_to_wo(word_to_tag, word_to_occurrence)

1781994it [00:05, 313529.66it/s]
23011601it [05:33, 69062.74it/s]
100%|██████████| 3591114/3591114 [00:12<00:00, 280381.54it/s]


# Generator

In [ ]:
def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

def choose_word_softmax(words):

    occurrences = np.array([word[1] for word in words], dtype=np.float64)

    occurrences = (occurrences - min(occurrences)) / (max(occurrences) - min(occurrences) + 0.0001)

    number_of_words = len(words)
    probs = softmax(occurrences)
    word_idx = np.random.choice(number_of_words , size = 1, p = probs)[0]

    return words[word_idx][0]

In [ ]:
def find_grammatically_similar_sentence_unigram(sen, word_to_tag, tag_to_words_with_occurrence):

    res = []

    for word in sen:

        if word in word_to_tag.keys():
            tag = word_to_tag[word]
        else:
            return "error :" + word

        res.append(choose_word_softmax(tag_to_words_with_occurrence[tag]))
    return res

In [ ]:
sen1 = 'Mały Piotruś spotkał w niewielkiej restauracyjce wczoraj poznaną koleżankę'.lower().split(' ')
sen2 = 'przed ćwiczeniami studenci dostarczają prowadzącemu deklaracje'.lower().split(' ')
sen3 = 'rycerze osaczyli smoka zanim strawił księżniczkę'.lower().split(' ')
sen4 = 'Przed wyruszeniem w drogę należy zebrać drużynę'.lower().split(' ')
sen5 = 'W zadaniu tym powinieneś losować zdania o słowach z identyczną charakterystyką gramatyczną jak zdanie wejściowe'.lower().split(' ')

In [ ]:
print(" ".join(sen1))
print(" ".join(find_grammatically_similar_sentence_unigram(sen1, word_to_tag, tag_to_words_with_occurrence)))

mały piotruś spotkał w niewielkiej restauracyjce wczoraj poznaną koleżankę
metalograficzny haszek sprofesjonalizował w nieprzejezdnej metrówce zbieracko rozpisaną lubekę


In [ ]:
print(" ".join(sen2))
print(" ".join(find_grammatically_similar_sentence_unigram(sen2, word_to_tag, tag_to_words_with_occurrence)))

przed ćwiczeniami studenci dostarczają prowadzącemu deklaracje
w usunięciami okupanci gotują wyświetlającemu wyobraźnie


In [ ]:
print(" ".join(sen3))
print(" ".join(find_grammatically_similar_sentence_unigram(sen3, word_to_tag, tag_to_words_with_occurrence)))

rycerze osaczyli smoka zanim strawił księżniczkę
nakładacze podciągnęli kocurka aniżeli liznął anastrofę


In [ ]:
print(" ".join(sen4))
print(" ".join(find_grammatically_similar_sentence_unigram(sen4, word_to_tag, tag_to_words_with_occurrence)))

przed wyruszeniem w drogę należy zebrać drużynę
przed schodzeniem ponad uchę asystuje uprościć fascjolozę


In [ ]:
print(" ".join(sen5))
print(" ".join(find_grammatically_similar_sentence_unigram(sen5, word_to_tag, tag_to_words_with_occurrence)))

w zadaniu tym powinieneś losować zdania o słowach z identyczną charakterystyką gramatyczną jak zdanie wejściowe
w wyklepaniu owym powinieneś holografować sczyszczenia o przezroczach ze kwaskowatą obrzędowością attykową jak upominanie frywolne


# Bigrams + Unigrams

In [ ]:
def load_bigrams_and_create_bitags_to_bigram_occurrence(path, word_to_tag, amount):
    
    file = open(path, 'r')
    bigramtag_to_bigram_occurrence = {}

    for iter, line in tqdm(enumerate(file)):

        oc, word1, word2 = line.lower().split(" ")

        if word2[-1] == "\n": word2 = word2[:-1]

        if word1 not in word_to_tag.keys() or word2 not in word_to_tag.keys(): continue
    
        if iter == amount: break

        key = (word_to_tag[word1], word_to_tag[word2])

        if key in bigramtag_to_bigram_occurrence.keys():
            bigramtag_to_bigram_occurrence[key].append(((word1, word2), int(oc)))
        else:
            bigramtag_to_bigram_occurrence[key] = [((word1, word2), int(oc))]     


    return bigramtag_to_bigram_occurrence

In [ ]:
bigrams_path = '/content/poleval_2grams.txt'
bitag_to_bigram_occurence = load_bigrams_and_create_bitags_to_bigram_occurrence(bigrams_path, word_to_tag, 24000000)

23982002it [02:13, 196773.50it/s]

# Generator

In [ ]:
import random

def find_grammatically_similar_sentence_bigram(sen, word_to_tag, bitag_to_bigram_occurence, tag_to_words_with_occurrence, recall):

    res = []
    
    err = 0
    while len(res) != len(sen):
        

        if len(res) < 2:
            res = []
            ### Generate first bigram
            word_1 = sen[0]
            word_2 = sen[1]

            if word_1 not in word_to_tag or word_2 not in word_to_tag: return "No tag for word"
            
            key = (word_to_tag[word_1], word_to_tag[word_2])
            
            pred_1, pred_2 = choose_word_softmax(bitag_to_bigram_occurence[key])
            res.append(pred_1)
            res.append(pred_2)

        len_res = len(res) - 1
            
        word_1 = sen[len_res]
        word_2 = sen[len_res+1]

        match_word = res[-1]
        if match_word[0] == '|': match_word = match_word[1:]

        if word_1 not in word_to_tag or word_2 not in word_to_tag: return "No tag for word"

        if err == recall:

            if random.random() < 0.3:   
                res.append('|' + choose_word_softmax(tag_to_words_with_occurrence[word_to_tag[word_2]]))
            else:
                res = res[:-2]
       
            err = 0
            continue

        key = (word_to_tag[word_1], word_to_tag[word_2])

        words = [((bigram[0], bigram[1]), int(oc)) for bigram, oc in bitag_to_bigram_occurence[key] if bigram[0]==match_word]

        if len(words) == 0:
            res = res[:-1]
            err += 1
        else:
            pred_1, pred_2 = choose_word_softmax(words)
            res.append(pred_2)

    
    return res   

In [ ]:
recall = 10
sen1 = 'Mały Piotruś spotkał w niewielkiej restauracyjce wczoraj poznaną koleżankę'.lower().split(' ')
sen2 = 'przed ćwiczeniami studenci dostarczają prowadzącemu deklaracje'.lower().split(' ')
sen3 = 'rycerze osaczyli smoka zanim strawił księżniczkę'.lower().split(' ')
sen4 = 'Przed wyruszeniem w drogę należy zebrać drużynę'.lower().split(' ')
sen5 = 'W zadaniu tym powinieneś losować zdania o słowach z identyczną charakterystyką gramatyczną jak zdanie wejściowe'.lower().split(' ')

In [ ]:
print(" ".join(sen1))
print(" ".join(find_grammatically_similar_sentence_bigram(sen1, word_to_tag, bitag_to_bigram_occurence, tag_to_words_with_occurrence, recall)))

mały piotruś spotkał w niewielkiej restauracyjce wczoraj poznaną koleżankę
godny następca podpisał w nietuzinkowej architekturze bardzo skrywaną tajemnicę


In [ ]:
print(" ".join(sen2))
print(" ".join(find_grammatically_similar_sentence_bigram(sen2, word_to_tag, bitag_to_bigram_occurence, tag_to_words_with_occurrence, recall)))

przed ćwiczeniami studenci dostarczają prowadzącemu deklaracje
między świętami zawodnicy przedkładają kupującemu akcje


In [ ]:
print(" ".join(sen2))
print(" ".join(find_grammatically_similar_sentence_bigram(sen2, word_to_tag, bitag_to_bigram_occurence, tag_to_words_with_occurrence, recall)))

przed ćwiczeniami studenci dostarczają prowadzącemu deklaracje
nad ustaleniami członkowie składają |świadczącemu prace


In [ ]:
print(" ".join(sen3))
print(" ".join(find_grammatically_similar_sentence_bigram(sen3, word_to_tag, bitag_to_bigram_occurence, tag_to_words_with_occurrence, recall)))

rycerze osaczyli smoka zanim strawił księżniczkę
obywatele stracili sokoła zanim spotkał ekspedycję


In [ ]:
print(" ".join(sen4))
print(" ".join(find_grammatically_similar_sentence_bigram(sen4, word_to_tag, bitag_to_bigram_occurence, tag_to_words_with_occurrence, recall)))

przed wyruszeniem w drogę należy zebrać drużynę
pomiędzy odżywianiem przed bramkę należy dofinansować różnicę


In [ ]:
print(" ".join(sen5))
print(" ".join(find_grammatically_similar_sentence_bigram(sen5, word_to_tag, bitag_to_bigram_occurence, tag_to_words_with_occurrence, recall)))

w zadaniu tym powinieneś losować zdania o słowach z identyczną charakterystyką gramatyczną jak zdanie wejściowe
na pomieszczeniu |którym powinieneś spodziewać rozpoczęcia o miejscach z godzinną normą równą jak zabezpieczanie antykorozyjne
